# augmentation을 적용한 함수 dl모델을 제작

데이터 증강 함수
- A.Resize(height=256, width=256) : 사이즈 변경
- A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) : 이미지 픽셀 정규화
- A.CoarseDropout(max_holes=16, max_height=16, max_width=16, min_holes=1, min_height=16, min_width=16) : 픽셀 드롭아웃(구멍뚫기)
- A.Rotate(limit=(115, 115), border_mode=cv2.BORDER_CONSTANT,value=[255, 255, 255], p=1.0) : 비율 유지하여 회전, 빈공간 흰색
- A.VerticalFlip(always_apply=False, p=0.5) : 좌우반전
- A.HorizontalFlip(always_apply=False, p=1.0) : 상하반전
- A.Blur(always_apply=True, p=1.0, blur_limit=(3, 7)) : 블러
- A.GaussNoise(always_apply=False, p=1.0, var_limit=(10.0, 50.0)) : 가우시안, 되는지 의문
- A.Downscale(always_apply=False, p=1.0, scale_min=0.35, scale_max=0.35, interpolation=0) : 픽셀다운, 모자이크
- 믹스 추가필요

데이터증강을 확률에 의한 적용 실험
생각 보다 성능이 나와주지 않음, 또한 tst set에 대해서도 데이터 증강이 적용 필요해 보임

In [1]:
import os
import time
import random

import timm
import torch
import albumentations as A
import pandas as pd
import numpy as np
import torch.nn as nn
from albumentations.pytorch import ToTensorV2
from torch.optim import Adam
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader, ConcatDataset
from PIL import Image
from tqdm import tqdm
from sklearn.metrics import accuracy_score, f1_score

import cv2
import matplotlib.pyplot as plt

from itertools import combinations

In [2]:
# 시드를 고정합니다.
SEED = 42
os.environ['PYTHONHASHSEED'] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.benchmark = True

In [3]:
# 데이터셋 클래스를 정의합니다.
class ImageDataset(Dataset):
    def __init__(self, csv, path, transform=None):
        self.df = pd.read_csv(csv).values
        self.path = path
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        name, target = self.df[idx]
        img = np.array(Image.open(os.path.join(self.path, name)))
        if self.transform:
            img = self.transform(image=img)['image']
        return img, target

In [4]:
# one epoch 학습을 위한 함수입니다.
def train_one_epoch(loader, vaild_loader, model, optimizer, loss_fn, device):
    model.train()
    train_loss = 0
    preds_list = []
    targets_list = []
    vaild_preds_list = []
    vaild_target_list = []

    pbar = tqdm(loader)
    for image, targets in pbar:
        image = image.float().to(device)
        targets = targets.to(device)

        model.zero_grad(set_to_none=True)

        preds = model(image)
        loss = loss_fn(preds, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())
        targets_list.extend(targets.detach().cpu().numpy())

        pbar.set_description(f"Loss: {loss.item():.4f}")

    train_loss /= len(loader)
    train_acc = accuracy_score(targets_list, preds_list)
    train_f1 = f1_score(targets_list, preds_list, average='macro')

    # vaildation set predict
    model.eval()
    for image, target in tqdm(vaild_loader):
        image = image.float().to(device)

        with torch.no_grad():
            preds = model(image)
        vaild_preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())
        vaild_target_list.extend(target.detach().cpu().numpy())
    vaild_acc = accuracy_score(vaild_target_list, vaild_preds_list)
    vaild_f1 = f1_score(vaild_target_list, vaild_preds_list, average='macro')

    ret = {
        "train_loss": train_loss,
        "train_acc": train_acc,
        "train_f1": train_f1,
        "vaild_acc": vaild_acc,
        "vaild_f1": vaild_f1,
    }

    return ret

In [5]:
# vaild_preds_list = []
# vaild_target_list = []

# model.eval()
# for image, target in tqdm(vaild_loader):
#     image = image.float().to(device)

#     with torch.no_grad():
#         preds = model(image)
#     vaild_preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())
#     vaild_target_list.extend(target.detach().cpu().numpy())
# vaild_acc = accuracy_score(vaild_target_list, preds_list)
# vaild_f1 = f1_score(vaild_target_list, preds_list, average='macro')

In [6]:
# device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# data config
data_path = '../datasets_fin/'

# model config
model_name = 'resnet34' # 'resnet50' 'efficientnet-b0', ...

# training config
img_size = 256
LR = 1e-3
EPOCHS = 100
BATCH_SIZE = 32
num_workers = 0

- 일단 단일 augmentation들로 확인
- 성능향상시 다양한 데이터증강을 조합하여 추가학습

In [10]:
# # augmentation을 위한 transform 코드
trn_transform = A.Compose([
    A.Resize(height=img_size, width=img_size),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2(),
])
trn_transform_coarseDropout = A.Compose([
    A.Resize(height=img_size, width=img_size),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    A.CoarseDropout(max_holes=30, max_height=16, max_width=16, min_holes=10, min_height=16, min_width=16, p=1),
    ToTensorV2(),
])
trn_transform_rotate45 = A.Compose([
    A.Resize(height=img_size, width=img_size),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    A.Rotate(limit=(45, 45), border_mode=cv2.BORDER_CONSTANT,value=[255, 255, 255], p=1),
    ToTensorV2(),
])
trn_transform_rotate90 = A.Compose([
    A.Resize(height=img_size, width=img_size),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]), 
    A.Rotate(limit=(90, 90), border_mode=cv2.BORDER_CONSTANT,value=[255, 255, 255], p=1),
    ToTensorV2(),
])
trn_transform_rotate135 = A.Compose([
    A.Resize(height=img_size, width=img_size),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    A.Rotate(limit=(135, 135), border_mode=cv2.BORDER_CONSTANT,value=[255, 255, 255], p=1),
    ToTensorV2(),
])
trn_transform_rotate180 = A.Compose([
    A.Resize(height=img_size, width=img_size),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    A.Rotate(limit=(180, 180), border_mode=cv2.BORDER_CONSTANT,value=[255, 255, 255], p=1),
    ToTensorV2(),
])
trn_transform_rotate225 = A.Compose([
    A.Resize(height=img_size, width=img_size),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    A.Rotate(limit=(225, 225), border_mode=cv2.BORDER_CONSTANT,value=[255, 255, 255], p=1.0),
    ToTensorV2(),
])
trn_transform_rotate270 = A.Compose([
    A.Resize(height=img_size, width=img_size),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    A.Rotate(limit=(270, 270), border_mode=cv2.BORDER_CONSTANT,value=[255, 255, 255], p=1.0),
    ToTensorV2(),
])
trn_transform_rotate315 = A.Compose([
    A.Resize(height=img_size, width=img_size),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    A.Rotate(limit=(315, 315), border_mode=cv2.BORDER_CONSTANT,value=[255, 255, 255], p=1.0),
    ToTensorV2(),
])
trn_transform_verticalflip = A.Compose([
    A.Resize(height=img_size, width=img_size),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    A.VerticalFlip(always_apply=False, p=1.0),
    ToTensorV2(),
])
trn_transform_horizontalflip = A.Compose([
    A.Resize(height=img_size, width=img_size),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    A.HorizontalFlip(always_apply=False, p=1.0),
    ToTensorV2(),
])
trn_transform_blur = A.Compose([
    A.Resize(height=img_size, width=img_size),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    A.Blur(always_apply=True, p=1.0, blur_limit=(3, 3)),
    ToTensorV2(),
])
trn_transform_gaussnoise = A.Compose([
    A.Resize(height=img_size, width=img_size),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    A.GaussNoise(always_apply=False, p=1.0, var_limit=(500.0, 500.0)),
    ToTensorV2(),
])
trn_transform_downscale = A.Compose([
    A.Resize(height=img_size, width=img_size),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    A.Downscale(always_apply=False, p=1.0, scale_min=0.5, scale_max=0.5, interpolation=0),
    ToTensorV2(),
])
transform_list = [trn_transform, trn_transform_coarseDropout, trn_transform_rotate45, trn_transform_rotate90, trn_transform_rotate135, trn_transform_rotate180, trn_transform_rotate225,
                  trn_transform_rotate270, trn_transform_rotate315, trn_transform_verticalflip, trn_transform_horizontalflip, trn_transform_blur, trn_transform_gaussnoise,
                  trn_transform_downscale]

# # test image 변환을 위한 transform 코드
# tst_transform = A.Compose([
#     A.Resize(height=img_size, width=img_size),
#     A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
#     ToTensorV2(),
# ])

In [11]:
trn_transform2 = A.Compose([
    A.Resize(height=img_size, width=img_size),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    A.CoarseDropout(max_holes=30, max_height=16, max_width=16, min_holes=10, min_height=16, min_width=16, p=0.5),
    # A.Rotate(limit=(45, 45), border_mode=cv2.BORDER_CONSTANT,value=[255, 255, 255], p=0.3),
    # A.Rotate(limit=(90, 90), border_mode=cv2.BORDER_CONSTANT,value=[255, 255, 255], p=0.3),
    # A.Rotate(limit=(135, 135), border_mode=cv2.BORDER_CONSTANT,value=[255, 255, 255], p=0.3),
    # A.Rotate(limit=(180, 180), border_mode=cv2.BORDER_CONSTANT,value=[255, 255, 255], p=0.3),
    # A.Rotate(limit=(225, 225), border_mode=cv2.BORDER_CONSTANT,value=[255, 255, 255], p=0.3),
    # A.Rotate(limit=(270, 270), border_mode=cv2.BORDER_CONSTANT,value=[255, 255, 255], p=0.3),
    # A.Rotate(limit=(315, 315), border_mode=cv2.BORDER_CONSTANT,value=[255, 255, 255], p=0.3),
    A.Rotate(limit=(0, 360), border_mode=cv2.BORDER_CONSTANT,value=[255, 255, 255], p=1),
    A.VerticalFlip(always_apply=False, p=0.5),
    A.HorizontalFlip(always_apply=False, p=0.5),
    A.Blur(always_apply=True, p=0.5, blur_limit=(3, 3)),
    A.GaussNoise(always_apply=False, p=0.5, var_limit=(500.0, 1000.0)),
    A.Downscale(always_apply=False, p=0.5, scale_min=0.5, scale_max=0.5, interpolation=0),
    ToTensorV2(),
])
tst_transform = A.Compose([
    A.Resize(height=img_size, width=img_size),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2(),
])

In [8]:
# transform_list에서 2개 조합으로 데이터 증강
# for i in combinations(transform_list, 2):
#     print(i, end="\n")

In [57]:
# Dataset 정의를 위한 함수
# def make_trn_dataset(transform_list, csv, path):
#     trn_dataset = ImageDataset(
#         csv,
#         path,
#         transform=transform_list[0]
#     )
#     for transform in transform_list[1:]:
#         trn_dataset2 = ImageDataset(
#             csv,
#             path,
#             transform=transform
#         )
#         trn_dataset = ConcatDataset([trn_dataset, trn_dataset2])

#     return trn_dataset

In [12]:
# vaildation set 정의
def make_vaild_dataset(transform_list, csv, path):
    trn_dataset = ImageDataset(
        csv,
        path,
        transform=transform_list[0]
    )
    for transform in transform_list[1:]:
        trn_dataset2 = ImageDataset(
            csv,
            path,
            transform=transform
        )
        trn_dataset = ConcatDataset([trn_dataset, trn_dataset2])

    return trn_dataset

In [59]:
# trn_dataset = make_trn_dataset(transform_list, "../datasets_fin/divided_train.csv", "../datasets_fin/train/")
# vaild_dataset = make_vaild_dataset(transform_list, "../datasets_fin/vaild.csv", "../datasets_fin/train/",)
# tst_dataset = ImageDataset(
#     "../datasets_fin/test.csv",
#     "../datasets_fin/train/",
#     transform=tst_transform
# )
# print(len(trn_dataset), len(vaild_dataset), len(tst_dataset))

17584 2198 157


In [13]:
# Dataset 정의를 위한 함수
trn_dataset = ImageDataset(
    "../datasets_fin/divided_train.csv",
    "../datasets_fin/train/",
    transform=trn_transform
)
vaild_dataset = make_vaild_dataset(transform_list, "../datasets_fin/vaild.csv", "../datasets_fin/train/")
tst_dataset = ImageDataset(
    "../datasets_fin/test.csv",
    "../datasets_fin/train/",
    transform=tst_transform
)
print(len(trn_dataset), len(vaild_dataset), len(tst_dataset))

1256 2198 157


In [14]:
# DataLoader 정의
trn_loader = DataLoader(
    trn_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    # shuffle=True,
    num_workers=num_workers,
    pin_memory=True,
    drop_last=False
)
vaild_loader = DataLoader(
    vaild_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    # shuffle=True,
    num_workers=num_workers,
    pin_memory=True,
    drop_last=False
)
tst_loader = DataLoader(
    tst_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=0,
    pin_memory=True
)

In [15]:
# load model
model = timm.create_model(
    model_name,
    pretrained=True,
    num_classes=17
).to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=LR)

In [16]:
early_stop = 0
best_model = model
best_epoch = 0
best_vaild_f1 = 0
for epoch in range(EPOCHS):
   if early_stop > 100: # earlystop없이 모든 애폭을 사용하여 학습, 단 best model을 따로 저장
      torch.save(best_model, '../model/best_augmentation3_'+str(img_size)+'_'+str(best_epoch)+'.pt')
      break
   ret = train_one_epoch(trn_loader, vaild_loader, model, optimizer, loss_fn, device=device)
   ret['epoch'] = epoch
   if best_vaild_f1 < ret['vaild_f1']:
      best_vaild_f1 = ret['vaild_f1']
      best_model = model
      best_epoch = epoch+1
      early_stop = 0
   else:
      early_stop += 1

   print(early_stop)

   log = ""
   for k, v in ret.items():
      log += f"{k}: {v:.4f}\n"
   print(log)

100%|██████████| 69/69 [00:11<00:00,  5.95it/s]


0
train_loss: 2.0199
train_acc: 0.4674
train_f1: 0.4329
vaild_acc: 0.4413
vaild_f1: 0.4257
epoch: 0.0000



100%|██████████| 69/69 [00:11<00:00,  5.95it/s]


0
train_loss: 0.5115
train_acc: 0.8479
train_f1: 0.8087
vaild_acc: 0.5159
vaild_f1: 0.5147
epoch: 1.0000



100%|██████████| 69/69 [00:11<00:00,  5.85it/s]


...

75
train_loss: 0.0001
train_acc: 1.0000
train_f1: 1.0000
vaild_acc: 0.4818
vaild_f1: 0.5257
epoch: 97.0000



100%|██████████| 69/69 [00:10<00:00,  6.34it/s]


76
train_loss: 0.0001
train_acc: 1.0000
train_f1: 1.0000
vaild_acc: 0.4850
vaild_f1: 0.5292
epoch: 98.0000



100%|██████████| 69/69 [00:10<00:00,  6.31it/s]

77
train_loss: 0.0001
train_acc: 1.0000
train_f1: 1.0000
vaild_acc: 0.4836
vaild_f1: 0.5291
epoch: 99.0000



In [40]:
# 베스트 모델 저장
torch.save(best_model, '../model/best_augmentation5_'+str(img_size)+'_'+str(best_epoch)+'.pt')

In [20]:
# 그냥 모델 저장
torch.save(model, '../model/augmentation5_'+str(img_size)+'_'+str(epoch+1)+'.pt')

In [16]:
# for epoch in range(20):
#     ret = train_one_epoch(trn_loader, vaild_loader, model, optimizer, loss_fn, device=device)
#     ret['epoch'] = epoch

#     log = ""
#     for k, v in ret.items():
#       log += f"{k}: {v:.4f}\n"
#     print(log)

100 번 학습한 모델의 데이터셋 확인 및 실제 데이터 추측

In [18]:
# test 데이터 확인
tst_preds_list = []
tst_target_list = []

model.eval()
for image, target in tqdm(tst_loader):
    image = image.float().to(device)

    with torch.no_grad():
        preds = model(image)
    tst_preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())
    tst_target_list.extend(target.detach().cpu().numpy())
tst_acc = accuracy_score(tst_target_list, tst_preds_list)
tst_f1 = f1_score(tst_target_list, tst_preds_list, average='macro')

tst_acc, tst_f1

100%|██████████| 5/5 [00:00<00:00,  5.80it/s]


(0.9363057324840764, 0.9339566873792841)

In [21]:
# 실제 test데이터
test_dataset = ImageDataset(
    "../datasets_fin/sample_submission.csv",
    "../datasets_fin/test/",
    transform=tst_transform
)
test_loader = DataLoader(
    test_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=0,
    pin_memory=True
)

In [22]:
preds_list = []

model.eval()
for image, _ in tqdm(test_loader):
    image = image.to(device)

    with torch.no_grad():
        preds = model(image)
    preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())

100%|██████████| 99/99 [00:13<00:00,  7.09it/s]


In [24]:
pred_df = pd.DataFrame(test_dataset.df, columns=['ID', 'target'])
pred_df['target'] = preds_list

In [25]:
sample_submission_df = pd.read_csv("../datasets_fin/sample_submission.csv")
assert (sample_submission_df['ID'] == pred_df['ID']).all()

In [26]:
pred_df.to_csv("../output/augmentation5_pred_256_"+str(epoch+1)+".csv", index=False)

In [27]:
pred_df.head()

,ID,target
0,0008fdb22ddce0ce.jpg,2
1,00091bffdffd83de.jpg,11
2,00396fbc1f6cc21d.jpg,8
3,00471f8038d9c4b6.jpg,11
4,00901f504008d884.jpg,2


best case의 test set확인 및 실제 데이터 추측

In [41]:
# 저장한 베스트 모델 로드
model = torch.load('../model/best_augmentation5_'+str(img_size)+'_'+str(best_epoch)+'.pt')

In [44]:
# test 데이터 확인
tst_preds_list = []
tst_target_list = []

model.eval()
for image, target in tqdm(tst_loader):
    image = image.float().to(device)

    with torch.no_grad():
        preds = model(image)
    tst_preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())
    tst_target_list.extend(target.detach().cpu().numpy())
tst_acc = accuracy_score(tst_target_list, tst_preds_list)
tst_f1 = f1_score(tst_target_list, tst_preds_list, average='macro')

tst_acc, tst_f1

100%|██████████| 5/5 [00:00<00:00,  7.09it/s]


(0.9363057324840764, 0.9339566873792841)

In [45]:
# 실제 test데이터
test_dataset = ImageDataset(
    "../datasets_fin/sample_submission.csv",
    "../datasets_fin/test/",
    transform=tst_transform
)
test_loader = DataLoader(
    test_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=0,
    pin_memory=True
)

In [46]:
preds_list = []

model.eval()
for image, _ in tqdm(test_loader):
    image = image.to(device)

    with torch.no_grad():
        preds = model(image)
    preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())

100%|██████████| 99/99 [00:14<00:00,  7.07it/s]


In [47]:
pred_df = pd.DataFrame(test_dataset.df, columns=['ID', 'target'])
pred_df['target'] = preds_list

In [48]:
sample_submission_df = pd.read_csv("../datasets_fin/sample_submission.csv")
assert (sample_submission_df['ID'] == pred_df['ID']).all()

In [49]:
pred_df.to_csv("../output/augmentation5_pred_256_"+str(best_epoch)+".csv", index=False)

In [50]:
pred_df.head()

,ID,target
0,0008fdb22ddce0ce.jpg,2
1,00091bffdffd83de.jpg,11
2,00396fbc1f6cc21d.jpg,8
3,00471f8038d9c4b6.jpg,11
4,00901f504008d884.jpg,2
